<a href="https://colab.research.google.com/github/Laurahg22/Trabajo-analitica-RH/blob/main/e_despliegue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
## Conectar el drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [109]:
import pandas as pd ### para manejo de datos
import sqlite3 as sql
import joblib
import openpyxl ## para exportar a excel
import numpy as np
import sys #para ver la ruta
from sklearn.preprocessing import LabelEncoder

sys.path ###Ruta directorio qué tiene paquetes
sys.path.append('/content/drive/MyDrive/trabajo/Trabajo-analitica-RH') ## este comanda agrega una ruta

import a_funciones as funciones  ###archivo de funciones propias

In [110]:
### funcion para cargar objeto guardado ###
df_2016_d =joblib.load("/content/drive/MyDrive/trabajo/Trabajo-analitica-RH/salidas/df_dummies_2016.pkl")

In [111]:
def columnas_nulos(df):
    # Obtener las columnas que tienen al menos un valor nulo
    columnas_con_nulos = df.columns[df.isnull().any()].tolist()
    return columnas_con_nulos

def imp_datos(df, variables):
    for variable in variables:
        # Calcula la moda de la variable
        moda = df[variable].mode()[0]  # Selecciona el primer valor de la moda en caso de que haya múltiples modas
        # Imputa los valores nulos con la moda
        df[variable].fillna(moda, inplace=True)
        # Imprime información sobre los valores nulos imputados
        nulos_imputados = df[variable].isnull().sum()
    # Devuelve el DataFrame modificado
    return df

def preparar_datos(df):

    list_cat = joblib.load("/content/drive/MyDrive/trabajo/Trabajo-analitica-RH/salidas/var_cat.pkl")
    list_dummies = joblib.load("/content/drive/MyDrive/trabajo/Trabajo-analitica-RH/salidas/list_dummies.pkl")
    var_names = joblib.load("/content/drive/MyDrive/trabajo/Trabajo-analitica-RH/salidas/var_names.pkl")
    scaler = joblib.load("/content/drive/MyDrive/trabajo/Trabajo-analitica-RH/salidas/scaler.pkl")

    nulos = columnas_nulos(df)
    df_t = imp_datos(df, nulos)
    le = LabelEncoder()
    for column in list_cat:
        if len(df_t[column].unique()) == 2:
            df_t[column] = le.fit_transform(df_t[column])
    df_t = pd.get_dummies(df_t)
    df_t = df_t.loc[:, ~df_t.columns.isin(['EmployeeID'])]

    # Asegurar que las dimensiones de los datos coincidan
    X2 = scaler.transform(df_t)  # Aplicar la transformación del scaler
    X = pd.DataFrame(X2, columns=df_t.columns)
    X = X[var_names]  # Seleccionar las variables necesarias

    return X


#Automatizar

In [ ]:
if __name__=="__main__":
    ## Cargar base de datos
    df_2016_d
    #### Otras transformaciones en python (imputación, dummies y selección de variables)
    df_t = preparar_datos(df_2016_d)

    ## Cargar modelo y predecir
    dt_final = joblib.load("/content/drive/MyDrive/trabajo/Trabajo-analitica-RH/salidas/dt_final.pkl")
    predicciones = dt_final.predict(df_t)
    pd_pred = pd.DataFrame(predicciones, columns=['Retiros'])

    ### Crear base con predicciones ###
    perf_pred = pd.concat([df_2016_d['EmployeeID'], df_t, pd_pred], axis=1)

    ####LLevar a BD para despliegue
    perf_pred[['EmployeeID', 'Retiros']].to_csv("/content/drive/MyDrive/trabajo/Trabajo-analitica-RH/salidas/perf_pred.csv", index=False)

    #### Ver_predicciones_bajas ###
    emp_pred_bajo = perf_pred.sort_values(by=["Retiros"], ascending=True).head(10)
    emp_pred_bajo.set_index('EmployeeID', inplace=True)
    pred = emp_pred_bajo.T
    coeficientes = pd.DataFrame(np.append([np.nan], [np.nan]), columns=['coeficientes'])  ### agregar coeficientes

    pred.to_excel("/content/drive/MyDrive/trabajo/Trabajo-analitica-RH/salidas/prediccion.xlsx")   #### exportar predicciones mas bajas y variables explicativas
    coeficientes.to_excel("/content/drive/MyDrive/trabajo/Trabajo-analitica-RH/salidas/coeficientes.xlsx") ### exportar coeficientes para analizar predicciones